## This notebook will use Long Short Term Memory (LSTM) to predict RUL for jet engines

In [5]:
import numpy as np

In [6]:
msg = "Roll a dice"
print(msg)

print(np.random.randint(1,9))

Roll a dice
4
